In [2]:
%load_ext autoreload
%autoreload 2

In [104]:
import collections
import concurrent.futures
import csv
import gc
import gzip
import glob
import itertools
import os
import sys
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pysam
import requests
from IPython.display import HTML

In [4]:
dir_project = os.path.abspath('..')
dir_modules = os.path.join(dir_project, 'modules')
dir_scripts = os.path.join(dir_project, 'scripts')
dir_data = os.path.join(dir_project, 'data')
dir_data_aux = os.path.join(dir_project, 'data_aux')
dir_data_dedup = os.path.join(dir_data, 'dedup')
os.makedirs(dir_data_dedup, exist_ok=True)

In [5]:
sys.path.append(dir_modules)
import utils_genomics

In [6]:
mouse_chromosomes_nuclear = [f'chr{i}' for i in range(1, 20)] + ['chrX', 'chrY']

In [12]:
directories_timecourse = sorted([x for x in os.listdir(os.path.join(dir_data, 'fastqs')) if x.endswith('minchase')], key=lambda s: int(s.split('minchase')[0]))

In [8]:
timepoints = sorted(map(lambda s: int(s.replace('minchase', '')), directories_timecourse))

In [9]:
columns_gtf = ['chr', 'source', 'feature type', 'start', 'end', 'score', 'strand', 'phase', 'additional']
columns_gff3 = ['chr', 'source', 'feature type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

# Check read structure

- Objective: identify any structural sequences from library preparation beyond standard Illumina TruSeq adapter sequences.
- Method
  1. Based on FASTQC's adapter content plot showing that Illumina adapter sequences start at ~50 nt into reads, extract all read pairs from the `0minchase` timepoint with the TruSeq adapter sequence present at exactly 50 nt into the forward *and* reverse reads.
     - This yielded 6157 paired reads.
  2. Perform multiple sequence alignment using [Kalign](https://www.ebi.ac.uk/Tools/msa/kalign/) on first 2000 sequences (maximum supported by EBI's Kalign service).
  3. Resulting alignment visualized using [MView](https://www.ebi.ac.uk/Tools/msa/mview/) did not show any "conserved" sequence.
- Result: no additional structural sequences identified

## Extract read pairs

In [116]:
def parse_fastq(fq):
    '''
    Parse fastq file. Argument is a file object opened in text mode.
    '''
    linecount = 0
    name, seq, third, qual = [None] * 4
    for linecount, line in enumerate(fq, start=1):
        if linecount % 4 == 1:
            name = line.rstrip()
            assert name.startswith('@')
        elif linecount % 4 == 2:
            seq = line.rstrip()
        elif linecount % 4 == 3:
            third = line.rstrip()
            assert third.startswith('+')
        elif linecount % 4 == 0:
            qual = line.rstrip()
            assert len(seq) == len(qual)
            yield name, seq, thrd, qual,
            name, seq, thrd, qual = [None] * 4

In [36]:
path_fastq_r1 = os.path.join(dir_data, 'fastqs', '0minchase', '10min5EU_0minchase_S3_R1_001.fastq.gz')
path_fastq_r2 = os.path.join(dir_data, 'fastqs', '0minchase', '10min5EU_0minchase_S3_R2_001.fastq.gz')

In [37]:
truseq_adapter_start = 'AGATCGGAAGAGC'

In [41]:
reads_idx50 = {}
with gzip.open(path_fastq_r1, 'rt') as f1, gzip.open(path_fastq_r2, 'rt') as f2:
    for read1, read2 in tqdm(zip(parse_fastq(f1), parse_fastq(f2))):
        name = read1[0].split()[0]
        assert name == read2[0].split()[0]
        idx = read1[1].find(truseq_adapter_start)
        if idx == 50 and idx == read2[1].find(truseq_adapter_start):
            reads_idx50[name] = (read1[1], read2[1])

0it [00:00, ?it/s]

In [42]:
len(reads_idx50)

6157

Reverse complement of insert sequence in read 2 is identical to insert sequence in read 1 (except in a few cases, presumably due to sequencing errors)

In [50]:
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
len(['a' for r1, r2 in reads_idx50.values() if r1[:50] == ''.join(complement.get(base) for base in reversed(r2[:50]))])

5995

## MSA

In [74]:
url_kalign = 'https://www.ebi.ac.uk/Tools/services/rest/kalign'
url_mview = 'https://www.ebi.ac.uk/Tools/services/rest/mview'

In [105]:
r = requests.post(
    url_kalign + '/run',
    data=dict(
        email='bentyeh@gmail.com',
        title='5eU_0minchase_msa',
        stype='dna',
        sequence='\n'.join([f'>{i}\n{v[0][:50]}' for i, v in enumerate(reads_idx50.values()) if i < 2000])))
jobid_kalign = r.content.decode()
print(jobid_kalign)

kalign-R20230409-085144-0196-44842580-p1m


In [107]:
print('status:', requests.get(url_kalign + f'/status/{jobid_kalign}').content.decode())

status: FINISHED


In [108]:
status = requests.get(url_kalign + f'/status/{jobid_kalign}').content.decode()
if status == 'FINISHED':
    msa = requests.get(url_kalign + f'/result/{jobid_kalign}/aln-clustalw').content.decode()

In [109]:
print(msa)

CLUSTAL multiple sequence alignment by Kalign (3.3.1)

0        -T-------------------------------------C-------A----A--C--G-
1        -G--------------------------C----------C-------C----C--C--G-
2        -A---------A-----------A----A----------A-------G----G--C--G-
3        -G---------------------C---CG----------C-------A----G--G--G-
4        -A--------------------------G----------C-------G----A--C--G-
5        -A---------A-----------A----A----------A-------G----G--C--G-
6        -C---------C---------G------C----------G-------G----C--T--G-
7        -AG--------G-----------G---TA----------G-------G----C--A--C-
8        -AG--------G-----------G---TA----------G-------G----C--A--C-
9        -C--------------------------G----------A-------G----C--A--G-
10       -G---------------------T----G----------C-------G----C-------
11       -C---------------------T----C----------A-------C----C--A--C-
12       -C--------------------------G----------A-------G----C--A--G-
13       -G---------------------G-A

In [111]:
r = requests.post(
    url_mview + '/run',
    data=dict(
        email='bentyeh@gmail.com',
        title='5eU_0minchase_msa_mview',
        stype='dna',
        alignment=True,
        width=120,
        sequence=msa.strip()))
jobid_mview = r.content.decode()
print(jobid_mview)

mview-R20230409-085353-0584-82258624-p1m


In [112]:
print('status:', requests.get(url_kalign + f'/status/{jobid_mview}').content.decode())

status: FINISHED


In [113]:
status = requests.get(url_mview + f'/status/{jobid_kalign}').content.decode()
if status == 'FINISHED':
    result_mview = requests.get(url_mview + f'/result/{jobid_mview}/out').content.decode()
    with open(os.path.join(dir_data_aux, 'reads_msa.html'), 'wt') as f:
        f.write(result_mview)

In [115]:
display(HTML(result_mview))

# Sequencing Alignment and QC

- Objective: count number of reads at each step of processing